In [16]:
import sys
import re
import pandas as pd
import numpy as np
import nltk
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from collections import Counter
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [17]:
nltk.download('stopwords')
from nltk import word_tokenize

y = pd.read_csv('novacontent.csv', header=0)
#to drop first column
y=y.iloc[:, 1:]
y.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dramakrishna/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Content
0,GET status 200
1,GET status 200
2,GET status 200
3,GET status 200
4,GET status 200


In [18]:
y['Content']=y['Content'].str.lower()
y['Content']=y['Content'].apply(word_tokenize)
#y.to_csv('content.csv')
y.to_csv('tokens.csv')

In [19]:
y.head(5)

,Content
0,"[get, status, 200]"
1,"[get, status, 200]"
2,"[get, status, 200]"
3,"[get, status, 200]"
4,"[get, status, 200]"


In [20]:
empty=[]
j=[]
for i in y['Content']:
    i_len = len(i)
    j=np.append(i,np.zeros(40-i_len))
    empty.append(j)


In [21]:
all_words=[]
for i in y['Content']:
    for j in i:
        all_words.append(j)

In [22]:
Counter(all_words)

Counter({'get': 274,
         'status': 275,
         '200': 274,
         '1': 35,
         'servers': 32,
         'os': 130,
         'interface': 7,
         'services': 15,
         'host': 7,
         'dbl031707': 1,
         'dbl': 4,
         'sys11cloud': 6,
         'net': 6,
         'binary': 5,
         'nova': 7,
         'compute': 6,
         'dbl031705': 1,
         'quota': 1,
         'sets': 1,
         'detail': 12,
         'name': 4,
         'worker': 2,
         '977ff8d75': 1,
         'zg59b': 1,
         'all_tenants': 1,
         'true': 1,
         'dbl021611': 1,
         'active': 1,
         'ewe': 1,
         'eam': 1,
         'exploration': 1,
         'mb7g7x': 1,
         '58754485d6': 1,
         '6mtn5': 1,
         'fes300409': 1,
         'fes': 2,
         'monitoring': 1,
         '669645f68f': 1,
         'gfhzz': 1,
         'virt': 1,
         'libvirt': 1,
         'imagecache': 1,
         'unknown': 1,
         'base': 2,
         'file

In [23]:
#Sentences are padded wit 0's to keep the length consistent in each row
empty[0]

array(['get', 'status', '200', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '0.0',
       '0.0', '0.0', '0.0', '0.0'], dtype='<U32')

In [29]:
onehot_enc = OneHotEncoder(sparse=False)
X = onehot_enc.fit_transform(empty)

In [30]:
onehot_enc.inverse_transform(X)

array([['get', 'status', '200', ..., '0.0', '0.0', '0.0'],
       ['get', 'status', '200', ..., '0.0', '0.0', '0.0'],
       ['get', 'status', '200', ..., '0.0', '0.0', '0.0'],
       ...,
       ['deprecated', 'in', '21', ..., '0.0', '0.0', '0.0'],
       ['new', 'default', 'roles', ..., '0.0', '0.0', '0.0'],
       ['new', 'default', 'roles', ..., '0.0', '0.0', '0.0']],
      dtype=object)

In [31]:
print(X[0])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.]


In [32]:

import random
random.shuffle(X)

In [35]:
if_model=IsolationForest(n_estimators=100, contamination=0.03)
if_model.fit(X)
print(if_model)

score_pred = if_model.decision_function(X)
#print(score_pred)
pred = if_model.predict(X)
#print(pred)
print(type(pred))

IsolationForest(contamination=0.03)
<class 'numpy.ndarray'>


In [36]:
arr = np.array([pred])
list1 = arr.tolist()
print(list1)
print(type(list1))
Counter(pred)

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Counter({1: 738, -1: 23})